## JOUNAL ENTRY TEST Version 2023
#### 아래 코드를 실행 시켜 주세요


# 사용설명
>  
>    1. GL, 휴일전표, ACCOUNT MAPPING파일을 모두 동일한 폴더에 위치시킵니다.
>    2. 위 코드를 실행합니다
>       - GL,휴일전표,Account Mapping 파일이 있는 폴더를 입력하세요 :: __아래 폴더 경로를 입력__
>       - 저장할 DB명 및 파일명을 입력하세요 :: __Ferring_Korea__
>    3. 코드가 종료되면 동일한 폴더 내의 JET_회사명으로 저장됩니다.
    
    

- - -
|회사명|파일위치|
|-----|-----|
|비에이치| \\\krsel0703\ASSG\190. Audit Analytics Solution_2018년 분석\Analytics Solution\92. 비에이치_JET\D-JET |
|CDNETWORKS|\\\krsel0703\ASSG\190. Audit Analytics Solution_2018년 분석\Analytics Solution\91. 씨디네트웍스\4000번\새 폴더|
|GDK|\\\krsel0703\ASSG\190. Audit Analytics Solution_2018년 분석\Analytics Solution\96. GDK\새 폴더|
|GSDK|\\\krsel0703\ASSG\190. Audit Analytics Solution_2018년 분석\Analytics Solution\94. GSDK|

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import time
import re
from sqlalchemy import create_engine
import os

def load_files(path):
    global gl
    global mapping
    global holi
    gl_file  = os.path.join(path, "GL.xlsx")
    h_file  = os.path.join(path,"Holiday_list.xlsx")
    mapping_file = os.path.join(path, "Accounting Mapping.xlsx")
    
    gl = pd.read_excel(gl_file)
    holi = pd.read_excel(h_file)
    mapping = pd.read_excel(mapping_file)

    ##--컬럼 타이틀에 엔터키 제거--##
    gl.columns = gl.columns.str.replace('\n', '') 
    holi.columns = holi.columns.str.replace('\n', '') 
    mapping.columns = mapping.columns.str.replace('\n', '') 
    ##--컬럼 타이틀에 엔터키 제거--##
    
def merge_holiday():
    global merge
    
    #------------------------merge two files------------------------##
    ##작성일자를 기준으로 GL과 휴일전표 통합##
    gl['작성일자'] = pd.to_datetime(gl['작성일자'])
    holi['날짜'] = pd.to_datetime(holi['날짜'])
    merge = gl.merge(holi, left_on='작성일자', right_on='날짜', how='left')
    #------------------------merge two files------------------------##
    
    ##---------------------데이터 유형변경---------------------##
    merge['작성일자'] = pd.to_datetime(merge['작성일자']).dt.date
    #merge['갱신일자'] = pd.to_datetime(merge['갱신일자']).dt.date
    merge['전기일자'] = pd.to_datetime(merge['전기일자']).dt.date
    #merge['증빙일자'] = pd.to_datetime(merge['증빙일자']).dt.date
    merge['계정과목코드'] = merge['계정과목코드'].astype(str)
    mapping['Account Number'] = mapping['Account Number'].astype(str)
    
    ##데이터 유형변경##
    #     merge['작성일자'] = np.where(merge['작성일자'].str.strip() == '','',pd.to_datetime(merge['작성일자']).dt.date)
    #     merge['갱신일자'] = np.where(merge['갱신일자'].str.strip() == '','',pd.to_datetime(merge['갱신일자']).dt.date)
    #     merge['전기일자'] = np.where(merge['전기일자'].str.strip() == '','',pd.to_datetime(merge['전기일자']).dt.date)
    #     merge['증빙일자'] = np.where(merge['증빙일자'].str.strip() == '','',pd.to_datetime(merge['증빙일자']).dt.date)
    ##---------------------데이터 유형변경---------------------##
    
    
    ##------------------------------------전처리 시작------------------------------------##
    merge['ABS(전표금액)'] = merge['전표금액(기준통화)'].abs() # 수정, 이승영, 2019-12-13
    
    #계정과목,작성자명,중복건수 count
    ###마지막 6자리숫자가 중복될 경우 - extract repeated digit
    regex = re.compile(r'(\d)\1{5}')
    merge['Digit'] = merge['ABS(전표금액)'].fillna(0).astype('int64').astype(str).apply(lambda x: x[-6:]) # 수정, 이승영, 2019-11-26
    merge['Ending Digit']  = np.where(merge['Digit'].str.match(regex),'O','X')
    
    # merge = merge.fillna(' ') # 수정, 이승영, 2019-12-09
    
    merge['사용빈도_계정과목(중복제외)']  =  np.where(merge.groupby(['회사코드','전표번호','계정과목코드']).cumcount() +1 == 1,1,0)
    merge['사용빈도_계정과목'] = merge.groupby(['계정과목코드','회사코드'])['사용빈도_계정과목(중복제외)'].transform('sum').replace(to_replace=0, method='ffill')
    merge['사용빈도_작성자명']  =  np.where(merge.groupby(['작성자ID','전표번호','회사코드']).cumcount() +1 == 1,1,0)
   
    merge['DuplicatedEntries'] = merge[['차대변구분자','전표금액(기준통화)','계정과목코드']].apply(lambda x : '{}{}{}'.format(x[0],x[1],x[2]), axis=1)
    merge['중복건수'] =merge.groupby('DuplicatedEntries')['DuplicatedEntries'].transform('count')
    
    #     전표적요가 공란인 경우 계산이 되지 않고 공란으로 표시되는 현상 수정 -- 2019-03-12
    #     전표적요가 숫자일 경우 공란으로 표시되는 현상 수정 -- 2019-03-12
    merge['적요길이'] =  merge['전표적요'].str.split().str.len()
    merge['적요길이'] =  merge['전표적요'].str.split().fillna('').str.len()
    
    
    
    ##전표금액(기준통화) 소숫점 5자리에서 합산
    merge['전표금액(기준통화)'] = merge['전표금액(기준통화)'].apply(lambda x: round(x, 5))
    
    #차대변 일치 혹은 null일 경우 O으로 대체-- 일치 하지 않을 경우 X 
    #- 모든 전표가 차대변 일치로 나오는 현상 수정 -- 2020-02-19
    merge['차대변일치test'] = merge.groupby(['전표번호'])['전표금액(기준통화)'].transform('sum')
    merge['차대변일치test'] = merge['차대변일치test'].astype(float).round(2).abs()
    merge['차대변일치'] = np.where(((merge['차대변일치test'] > 0) | (merge['차대변일치test'] < 0)),'X','O')
    
    ##전표금액(기준통화) 소숫점 5자리에서 합산
    merge['전표금액(기준통화)'] = merge['전표금액(기준통화)'].apply(lambda x: round(x, 5))
    
    ##전표행번,전표번호 같은전표가 다른날짜에 있는경우
    merge['중복전표test']  =  np.where(merge.groupby(['전기일자','전표번호']).cumcount() +1 == 1,1,0)
    merge['중복전표test2'] = merge.groupby(['전표번호'])['중복전표test'].transform('sum').replace(to_replace=0, method='ffill')
    merge['중복전표']  = np.where(merge['중복전표test2'] == 1,'X','O')
    
    
    merge['작성일-전기일'] = pd.Series(delta.days for delta in (merge['전기일자'] - merge['작성일자'])).abs()
    ##2019-02-27 수정 날짜가 NaN값일 경우
    #merge['작성일-전기일'] = merge['작성일자'].astype(dt.timedelta).map(lambda x:np.nan if pd.isnull(x) else x.day).sub(merge['전기일자'].astype(dt.timedelta).map(lambda x:np.nan if pd.isnull(x) else x.day)).abs()
    merge['전표-차변금액합계'] = merge.groupby('전표번호')['차변금액(기준통화)'].transform('sum')
    merge['전표-차변금액합계(RANK)'] =merge['ABS(전표금액)'].rank(ascending=0,method='min') 
    ##------------------------------------전처리 끝------------------------------------##
    

def merge_mapping():
    
    global final_merge
    
    ######---------------------------------------Final merge-------------------------------------------------------------
    ##Account Mapping과 통합
    final_merge = merge.merge(mapping, left_on='계정과목코드', right_on='Account Number', how='left')
    ##최종통합 테이블에서 NULL값 제거
    final_merge = final_merge.fillna('')
    final_merge['FS Line'] = final_merge['FS Line'].astype(str)
    final_merge['Account Subtype'] = final_merge['Account Subtype'].astype(str) 
    final_merge['차대변구분자'] = final_merge['차대변구분자'].astype(str)
    final_merge['차변금액(기준통화)'] = final_merge['차변금액(기준통화)'].replace('',0).astype(np.int64)
    final_merge['대변금액(기준통화)'] = final_merge['대변금액(기준통화)'].replace('',0).astype(np.int64)
    
    ####------------------------계정구분-차/대 == C와 D를 스위치- 시작--------------------------------------------------------------------------####
    final_merge['계정구분-차/대'] = np.where(final_merge['FS Line'],final_merge['FS Line'] + "-"+
np.where(np.logical_and(final_merge['차변금액(기준통화)'] + final_merge["대변금액(기준통화)"] <0
                                            ,final_merge['차대변구분자']=='C'),'D',
                            np.where(np.logical_and(final_merge['차변금액(기준통화)'] + final_merge["대변금액(기준통화)"] <0
                                            ,final_merge['차대변구분자']=='D'),'C',final_merge['차대변구분자']) ),'')

    ####------------------------계정구분-차/대 == C와 D를 스위치 끝---------------------------------------------------------------------------####
    final_merge['calc'] = final_merge.groupby(['전표번호','계정구분-차/대',])['전표금액(기준통화)',].transform('sum')
    ## ----group에서 계정구분 차/대 Revenue-C를 구한 후 그 값을 그룹내에 다른 전표에도 전부적용 2019-1-14 ---------------##
    final_merge['Revenue-C']= np.where (final_merge['계정구분-차/대'] == 'Revenue-C', final_merge['calc'],np.nan)
    final_merge['Revenue-D']= np.where (final_merge['계정구분-차/대'] == 'Revenue-D', final_merge['calc'],np.nan)
    final_merge['Cost of sales-C']= np.where (final_merge['계정구분-차/대'] == 'Cost of sales-C', final_merge['calc'],np.nan)
#아래계산부분 2023-12-15 update
    groupby_result = final_merge.groupby(['전표번호'])['Revenue-C'].apply(lambda x: x.ffill().bfill())
    final_merge['매출_대변합계/전표'] = groupby_result.reset_index(level=0, drop=True)
    groupby_result = final_merge.groupby(['전표번호'])['Revenue-D'].apply(lambda x: x.ffill().bfill())
    final_merge['매출_차변합계/전표']=  groupby_result.reset_index(level=0, drop=True)
    groupby_result = final_merge.groupby(['전표번호'])['Cost of sales-C'].apply(lambda x: x.ffill().bfill())
    final_merge['매출원가_대변합계/전표']=  groupby_result.reset_index(level=0, drop=True)

    final_merge['Account Subtype-차/대'] = np.where(final_merge['Account Subtype'] ,final_merge['Account Subtype'] + "-"+ final_merge['차대변구분자'],'')
    final_merge['Account Subtype-차/대 합계'] = final_merge.groupby(['전표번호','Account Subtype',])['전표금액(기준통화)'].transform('sum')
    final_merge['사용빈도_작성자명_합계'] = final_merge.groupby('작성자ID')['사용빈도_작성자명'].transform('sum')
    final_merge['적요길이'] =  final_merge['전표적요'].str.split().fillna(' ').str.len()

    ##--------------------------------사용 컬럼만 적용------------------------------------##
    columns=['회사코드', '회사명', '전표번호', '작성일자', '작성시간', '갱신일자', '갱신시간', '작성자ID', '작성자명','갱신자ID', '갱신자명', '전기일자', 
    '증빙일자', '회계월', '전표유형','전표유형명', '자동수동구분','전표적요', '전표행번', '전표적요상세', '전표통화코드', '기준통화코드', '환율', '차대변구분자', 
    '전표금액','차변금액', '대변금액','전표금액(기준통화)', '차변금액(기준통화)', '대변금액(기준통화)','계정과목코드', '계정과목명', 'Cost Center 코드', 'Cost Center 명','Profit Center 코드', 'Profit Center 명', 
    '원천시스템 명(또는 거래 코드명)','ABS(전표금액)','중복전표','차대변일치','주말/휴일','Ending Digit','사용빈도_계정과목(중복제외)','사용빈도_계정과목','사용빈도_작성자명',
             '적요길이','작성일-전기일','전표-차변금액합계','전표-차변금액합계(RANK)','Account Subtype', 'FS Line',
             '계정구분-차/대','매출_대변합계/전표','매출_차변합계/전표','매출원가_대변합계/전표','사용빈도_작성자명_합계',
             'Account Subtype-차/대','Account Subtype-차/대 합계' ]
    final_merge = final_merge[columns]
    ##--------------------------------------사용 컬럼만 적용------------------------------------##

def save_to_Excel(Saved_Name):
    output_file = ''.join([path, "/" , Saved_Name,".xlsx"])
    final_merge.to_excel(output_file,index=False)
    
if __name__ == "__main__":
    print("Start Time "+ time.strftime('%Y-%m-%d %H:%M:%S'))
    print("--------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("-----------------------------------GL,Holiday_list,Accounting Mapping 파일은 동일한 폴더안에 있어야 합니다.------------------------------------------------")
    print("--------------------------------------------------------------------------------------------------------------------------------------------------------")
    path = input("GL,휴일전표,Account Mapping 파일이 있는 폴더를 입력하세요 :: ")
    print("--------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("Loading Files... @ " + time.strftime('%H:%M:%S'))
    load_files(path)
    print("Merging with Holiday_list... @ " + time.strftime('%H:%M:%S'))
    merge_holiday()
    print("Merging with Accounting Mapping... @ " + time.strftime('%H:%M:%S'))
    merge_mapping()
    SavedFile = input("저장할 파일명을 입력하세요 :: ")
    Saved_Name = "JET_"+ SavedFile 
    print("Saving Excel as... " + Saved_Name + " @ " + time.strftime('%H:%M:%S'))
    save_to_Excel(Saved_Name)
    print("End Time "+ time.strftime('%Y-%m-%d %H:%M:%S'))

Start Time 2023-12-15 14:01:29
--------------------------------------------------------------------------------------------------------------------------------------------------------
-----------------------------------GL,Holiday_list,Accounting Mapping 파일은 동일한 폴더안에 있어야 합니다.------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------
GL,휴일전표,Account Mapping 파일이 있는 폴더를 입력하세요 :: C:\Users\kyobkoo\Desktop\바탕화면정리\ASSG\Excel Jet\2023\씨엔에스에너지\씨앤에스에너지 엑셀JET\GL setting
--------------------------------------------------------------------------------------------------------------------------------------------------------
Loading Files... @ 14:01:34


C:\Users\kyobkoo\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Merging with Holiday_list... @ 14:01:56
Merging with Accounting Mapping... @ 14:01:58
저장할 파일명을 입력하세요 :: GL
Saving Excel as... JET_GL @ 14:02:06
End Time 2023-12-15 14:04:02
